https://www.kaggle.com/tuckerarrants/text-generation-with-huggingface-gpt2

In [ ]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 8.7 MB/s 
     |████████████████████████████████| 596 kB 79.8 MB/s 
     |████████████████████████████████| 59 kB 8.4 MB/s 
     |████████████████████████████████| 3.3 MB 74.9 MB/s 
     |████████████████████████████████| 895 kB 82.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone  https://github.com/sberbank-ai/ru-gpts

Cloning into 'ru-gpts'...
remote: Enumerating objects: 663, done.
remote: Counting objects: 100% (158/158), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 663 (delta 97), reused 127 (delta 76), pack-reused 505
Receiving objects: 100% (663/663), 396.66 KiB | 6.72 MiB/s, done.
Resolving deltas: 100% (397/397), done.


In [ ]:
import numpy as np
import pandas as pd
import re
import random

import torch
from tqdm.notebook import tqdm
import transformers

if torch.cuda.is_available():    
    device = torch.device("cuda")
else:
    device = torch.device("cpu")


In [ ]:
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('sberbank-ai/rugpt3small_based_on_gpt2')

Downloading:   0%|          | 0.00/1.63M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/608 [00:00<?, ?B/s]

In [ ]:
import re
with open('/content/drive/MyDrive/NLP_Project.Harry_Potter_AI/Russian/v2/Copy of merged_books.txt', encoding='utf8') as f:
  text = f.read().replace('.', '.\n').replace('?', '?\n').replace('!', '!\n').split('\n')
  #text = re.sub('\n{2,}', '\n', text)
  text = [re.sub('[0-9]', '', i) for i in text]
  text = [re.sub('\]', '', i) for i in text]
  text = [re.sub('\[', '', i) for i in text]
  text = [i.strip() for i in text]
print(text[1:31])

['Мистер и миссис Дурсль проживали в доме номер четыре по Тисовой улице и всегда с гордостью заявляли, что они, слава богу, абсолютно нормальные люди.']


In [ ]:
train = []
test = []
max_length = 0
index_max_length = 0

for idx, value in enumerate(text):
  tokens = tokenizer.encode(value, add_special_tokens=True)
  tokens = np.array(tokens)

  curr_len = len(tokens)
  if curr_len >= max_length:
    max_length = curr_len
    index_max_length = idx

  if idx <= (len(text) * .8):
    train.append(tokens)

  else:
    test.append(tokens)

train = np.array(train)
test = np.array(test)

print('len(train), len(test): ', len(train), len(test))
print('max_length, index_max_length: ', max_length, index_max_length)

len(train), len(test):  111693 27923
max_length, index_max_length:  183 88408


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [ ]:
def Padding(review_int, seq_len):
    '''
    Return features of review_ints, where each review is padded with 0's or truncated to the input seq_length.
    '''
    features = np.zeros((len(review_int), seq_len), dtype = int)
    for i, review in enumerate(review_int):
        if len(review) <= seq_len:
            zeros = list(np.zeros(seq_len - len(review)))
            # print(len(zeros))
            # print(review.shape)
            # new = np.array(zeros) + review
            new = np.append(zeros, review)

        features[i, :] = np.array(new)
            
    return features

train = Padding(train, 150)
test = Padding(test, 150)

print(train.shape)
print(train[:1])

(111693, 150)
[[    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0 31711   312
   2171   842   289  8060   778  8781]]


In [ ]:
# tokens = tokenizer.encode(text, add_special_tokens=True)
# tokens = np.array(tokens)
# print(len(tokens))
# tokens[:10]

1298892


array([31711,   312,  2171,   842,   289,  8060,   778,  8781,   203,
       10594])

In [ ]:
# l = len(tokens)//15
# train = []
# test = []
# for i in range(15):
#     if i%5 > 0:
#         train.extend(tokens[i*l: (i+1)*l])
#     else:
#         test.extend(tokens[i*l: (i+1)*l])
# train = np.array(train)
# test = np.array(test)

# print(len(tokens), len(train), len(test))

1298892 1039104 259776


In [ ]:
from transformers import GPT2LMHeadModel, AdamW

model_init = GPT2LMHeadModel.from_pretrained(
    'sberbank-ai/rugpt3small_based_on_gpt2',
    output_attentions = False,
    output_hidden_states = False,
)


model = GPT2LMHeadModel.from_pretrained(
    'sberbank-ai/rugpt3small_based_on_gpt2',
    output_attentions = False,
    output_hidden_states = False,
)

model.to(device);
model_init.to(device);

Downloading:   0%|          | 0.00/526M [00:00<?, ?B/s]

In [ ]:
batch_size = 12
#max_len = 256
epochs = 7

n_train = len(train)//(batch_size+1) #*max_len)
n_test = len(test)//(batch_size+1) #*max_len)
print(n_train, n_test)

optimizer = AdamW(model.parameters(), lr = 1e-5, eps = 1e-8)

total_steps = n_train * epochs
scheduler = transformers.get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)


def accuracy(y_true, logits):
    return torch.mean((y_true[1:] == torch.argmax(logits, dim=2)[:-1]).float()).detach().cpu().numpy()

8591 2147


In [ ]:
# def prep_tensors(x, i, batch_size=batch_size, max_len=max_len):
#     batch_ids = x[i*batch_size*max_len: (i+1)*batch_size*max_len]
#     batch_ids = batch_ids.reshape(batch_size, max_len)
#     batch_ids = torch.tensor(batch_ids).to(device)
#     return batch_ids


def prep_tensors(x, i, batch_size=batch_size):
    start_idx = i*batch_size
    end_idx = start_idx + batch_size
    batch_ids = x[start_idx: end_idx]
    batch_ids = torch.tensor(batch_ids).to(device)
    return torch.tensor(batch_ids).to(device)


preped = prep_tensors(train, 17)
print('preped shape: ', preped.shape)

for epoch in range(1, epochs+1):
    print(f'epoch {epoch}/{epochs} : training')

    train_loss = []
    train_acc = []
    model.train()
    pbar = tqdm(range(n_train))
    for i in pbar:
        batch_ids = prep_tensors(train, i)

        model.zero_grad()
        loss, logits, _ = model(batch_ids,
                             token_type_ids=None, 
                            #  attention_mask=batch_mask,
                             labels=batch_ids
                             ).values()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        
        train_loss.append(loss.item())
        train_acc.append(accuracy(batch_ids, logits))
        pbar.set_description(f'acc {np.mean(train_acc):.4f} loss {np.mean(train_loss):.4f}', refresh=True)

    
    print('epoch {epoch}/{epochs} : validation')
    model.eval()
    val_acc = []
    val_loss = []
    pbar = tqdm(range(n_test))
    for i in pbar:
        batch_ids = prep_tensors(test, i)
        with torch.no_grad():        
            loss, logits, _ = model(batch_ids, 
                                token_type_ids=None, 
                                # attention_mask=batch_mask,
                                labels=batch_ids
                                 ).values()
        
        val_loss.append(loss.item())
        val_acc.append(accuracy(batch_ids, logits))
        pbar.set_description(f'acc {np.mean(val_acc):.4f} loss {np.mean(val_loss):.4f}', refresh=True)


preped shape:  torch.Size([12, 150])
epoch 1/7 : training


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]


  0%|          | 0/8591 [00:00<?, ?it/s]

epoch {epoch}/{epochs} : validation


  0%|          | 0/2147 [00:00<?, ?it/s]

...

  0%|          | 0/2147 [00:00<?, ?it/s]

epoch 7/7 : training


  0%|          | 0/8591 [00:00<?, ?it/s]

epoch {epoch}/{epochs} : validation


  0%|          | 0/2147 [00:00<?, ?it/s]

In [ ]:
#Save the model to a pkl or something so it can be reused later on
torch.save(model, '/content/drive/MyDrive/HP_gen_v.2_model_ru.pt')

In [ ]:
#Load the model to use it
model_first = torch.load('/content/drive/MyDrive/model_ru.pt')

In [ ]:
  # https://huggingface.co/transformers/main_classes/model.html#transformers.generation_utils.GenerationMixin.generate
# Model without training
import textwrap

# prompt = "print('Hello world!') "
prompt = 'Малфой обернулся и сильно'
prompt = tokenizer.encode(prompt, return_tensors='pt').to(device)
out = model.generate(
    input_ids=prompt,
    max_length=150,
    num_beams=7,
    do_sample=True,
    temperature=2.,
    top_k=50,
    top_p=0.7,
    no_repeat_ngram_size=2,
    num_return_sequences=1,
    ).cpu().numpy()
# for out_ in out:
#   print(textwrap.fill(tokenizer.decode(out_), 120), end='\n------------------\n')
mes = textwrap.fill(tokenizer.decode(*out), 120)
print(mes)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:2142: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size


Малфой обернулся и сильно толкнул Рона локтем, так что у того издала неприятный хруст, а Гарри и Джинни едва не
свалились с стульев. Рон и Гермиона исчезли в неизвестном направлении. Оставшись один, Гарри повернулся к двери и
крикнул: — Фред и Джордж, скорее сюда, в вашу спальню, я хочу поговорить с вами насчёт того, что произошло вчера
вечером, прежде чем вы вернётесь в «Три метлы» к обеду… ну, как вам будет угодно, но не могли бы вы пойти со мной на
обед в кабинет мистера Уизли, сэр, и сказать мне, где сейчас находится профессор Трелони, если вы не хотите попасться ей
на глаза и что вы собираетесь делать, когда попадётесь мне


In [ ]:
import textwrap

prompt = 'Малфой обернулся и сильно'
prompt = tokenizer.encode(prompt, return_tensors='pt').to(device)
out = model.generate(
    input_ids=prompt,
    max_length=150,
    num_beams=10,
    do_sample=True,
    temperature=2.,
    top_k=50,
    top_p=0.5,
    no_repeat_ngram_size=2,
    num_return_sequences=7,
    ).cpu().numpy()
for out_ in out:
    print(textwrap.fill(tokenizer.decode(out_), 120), end='\n------------------\n')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:2142: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size


Малфой обернулся и сильно ударил Гарри по затылку, так что тот едва не свалился с метлы на землю вместе со стулом Гарри.
Потом поднял голову и посмотрел на Гарри: лицо его было белое, глаза закрыты. Глаза были закрыты, волосы всклокочены, в
руке он держал длинный чёрный зонтик, который, судя по всему, был сделан из чёрной жести — такого же цвета, как и её
владелец, но зато гораздо более тяжёлый, чем тот, что был у Гарри в прошлом году на уроке зельеварения у Дамблдора,
когда они с Хагридом стояли на платформе Гриффиндора и махали ему на прощанье, прежде чем покинуть платформу вместе с
чемоданом, на котором они ехали в Хогвар
------------------
Малфой обернулся и сильно ударил Гарри по руке, отчего тот чуть не упал, больно стукнувшись спиной об пол, и упал на
колени рядом с ним. Гарри открыл глаза и увидел профессора Дамблдора. Они стояли друг против друга и смотрели друг другу
в глаза: профессор Дамблдор и профессор Люпин — оба были в чёрной мантии, одна из которых была сшита из тёмно-

In [ ]:
import textwrap

prompt = 'Малфой обернулся и сильно'
prompt = tokenizer.encode(prompt, return_tensors='pt').to(device)
out = model_init.generate(
    input_ids=prompt,
    max_length=150,
    num_beams=10,
    do_sample=True,
    temperature=2.,
    top_k=50,
    top_p=0.5,
    no_repeat_ngram_size=2,
    num_return_sequences=7,
    ).cpu().numpy()
for out_ in out:
    print(textwrap.fill(tokenizer.decode(out_), 120), end='\n------------------\n')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:2142: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size


Малфой обернулся и сильно сжал руку.  — Что случилось, Гарри? — спросил он, не поворачиваясь к нему лицом. Он не был
уверен в том, что Гарри смотрит на него так, словно это был не человек, а какая-то диковинная птица, но в то же время в
нем было нечто такое… что он не мог не узнать. Гарри не знал, как это назвать, и не понимал, почему ему вдруг пришло в
голову это имя. Может быть, он просто не хотел, чтобы ему напоминали об этом человеке? Или же ему показалось, будто он
узнал его, когда тот был в последний раз в кабинете Дамблдора. Но что это за человек? И что ему вообще известно о Гарри
Поттере?
------------------
Малфой обернулся и сильно ударил его по лицу.  – Я не хочу, чтобы ты пострадал, – тихо проговорил он. Он был в ярости, и
его лицо было искажено от боли, но он не обращал внимания на боль, потому что чувствовал, что она причиняет ему сильную
боль. Но в то же время он чувствовал себя очень усталым и измотанным, так как ему нужно было что-то предпринять. И,
несмотря на это, он 

In [ ]:
import textwrap

prompt = 'Малфой обернулся и сильно'
prompt = tokenizer.encode(prompt, return_tensors='pt').to(device)
out = model_first.generate(
    input_ids=prompt,
    max_length=150,
    num_beams=10,
    do_sample=True,
    temperature=2.,
    top_k=50,
    top_p=0.5,
    no_repeat_ngram_size=2,
    num_return_sequences=7,
    ).cpu().numpy()
for out_ in out:
    print(textwrap.fill(tokenizer.decode(out_), 120), end='\n------------------\n')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:2142: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size


Малфой обернулся и сильно ударил Гарри кулаком в лицо. — Да как ты смеешь! Гарри не знал, что и сказать, он стоял как
вкопанный, не в силах вымолвить ни слова. — Не смей, Гарри! Ты же знаешь, как я его люблю, и я… Но он уже не слышал. Он
смотрел на Рона и Гермиону. Они стояли и смотрели на него, покачиваясь в такт музыке, которая звучала в их головах,
когда они поднимались по лестнице в башню Гриффиндора. Гарри не мог заставить себя произнести слово «Люциус», он не
хотел говорить о Малфое, потому что знал: Малфой никогда ему не поверит. Но если бы только он был уверен,
------------------
Малфой обернулся и сильно ткнул в Малфоя волшебной палочкой. — Убирайтесь отсюда! — завопил Малфой, но тут же умолк,
увидев, что Гарри, Рон, Гермиона и Невилл стоят, прислонясь к стене, а за ними стоит профессор Флитвик, который, по-
видимому, ещё не пришёл в себя от потрясения, случившегося с ним в ту минуту, когда он увидел, как он упал с метлы. Он
был бледен, волосы спутаны, лицо бледное и осунувшее

In [ ]:
def generate(prompt, len_gen=20, temperature=1):
    generated = tokenizer.encode(prompt)
    context = torch.tensor([generated]).to(device)
    past = None

    for i in tqdm(range(len_gen)):
        output, past = model_2(context, past_key_values=past).values()
        # token = torch.argmax(output[..., -1, :], dim=-1)
        output = output / temperature
        token = torch.distributions.Categorical(logits=output[..., -1, :]).sample()
        
        generated += token.tolist()
        context = token.unsqueeze(0)

    sequence = tokenizer.decode(generated)

    return sequence

In [ ]:
import textwrap


# prompt = """<!DOCTYPE html>
# <html>
# <body>

# <h1>My First Heading</h1>
# <p>My first paragraph.</p>"""
prompt = 'Harry'
print(textwrap.fill(generate(prompt, 300, temperature=1), 120))

  0%|          | 0/300 [00:00<?, ?it/s]

Harry some of their gloomy portraits from the port told Harry that this wasn't their plan, as Mirrorback had fixed its
spirits on him seven years.   Madame Pomfrey, who had always booked tickets for the duel, gave a jaunty roging laugh.
Harry grinned broadly at the portrait in front of him, and, watching his master, turned up his luck. His blackened head
nudged however that Mr. Catalinoy was trudging toward the castle entrance. Nevertheless, Harry put on his Malfoy
mustache and started peering around to ensure that all the dining rooms were properly blank. The skies told of a great,
bright morning with a couple of trolls. “Right then, let’s get going,” said Dumbledore. “I need sniggerin” “Right then?”
Harry watched his master quietly stumble into Mirrorback’s dingeballs, a cauldron full of potion powder to suppress his
reading and he began to toast a table bowl of thick, spiced unbleeding honey.  Hermione was talking, but there was
laughter. Foggy water bubbled behind her ashtrays unde